# Diagnostiquez automatiquement des radios thoraciques.

Maintenant que vous avez découvert les CNN et que vous savez entraîner des modèles avec Pytorch, vous allez pouvoir utiliser vos nouvelles connaissances pour résoudre un problème plus concret.

Dans ce notebook, vous allez entraîner un algorithme capable de diagnostiquer automatiquement si notre patient est atteint d'une pneumonie ou non grâce à sa radiographie thoracique. Les données viennent de ce [datasets](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia) de Kaggle.


# Importation des packages

In [ ]:
import matplotlib.pyplot as plt

import os
import numpy as np

from sklearn.metrics import accuracy_score, confusion_matrix

import torch as torch
from torchvision import datasets, transforms
from torchsummary import summary
from collections import OrderedDict

from torch.utils.data import Subset

# Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1F1MiX9qQ7ZoafVq68x6ISWgtOsZf7CCi?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Visualisons nos données !

In [ ]:

input_path = '/content/drive/MyDrive/chest_xray/'

fig, ax = plt.subplots(2, 3, figsize=(15, 7))
ax = ax.ravel()
plt.tight_layout()

for i, _set in enumerate(['train', 'val', 'test']):
    set_path = input_path+_set
    ax[i].imshow(plt.imread(set_path+'/NORMAL/'+os.listdir(set_path+'/NORMAL') [i]), cmap='gray')
    ax[i].set_title('Set: {}, Condition: Normal'.format(_set))
    ax[i+3].imshow(plt.imread(set_path+'/PNEUMONIA/'+os.listdir(set_path+'/PNEUMONIA') [i]), cmap='gray')
    ax[i+3].set_title('Set: {}, Condition: Pneumonia'.format(_set))

In [ ]:
normal_example = os.listdir(f'{input_path}train/NORMAL')[3]
pneumonia_example = os.listdir(f'{input_path}train/PNEUMONIA')[3]

In [ ]:
normal_img = plt.imread(f'{input_path}train/NORMAL/{normal_example}')
plt.imshow(normal_img)
plt.show()

In [ ]:
pneumonia_img = plt.imread(f'{input_path}train/PNEUMONIA/{pneumonia_example}')
plt.imshow(pneumonia_img)
plt.show()

In [ ]:
test_labels = []
for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + 'test' + cond)):
          if cond=='/NORMAL/':
              label = 0
          elif cond=='/PNEUMONIA/':
              label = 1
          test_labels.append(label)

test_labels = np.array(test_labels)

# Initialiser les générateurs

Utilisez la classe *transforms* pour appliquer des transformation à vos images. C'est cette classe qui est aussi utilisez pour effectuer de la data augmentation.

Appliquez :
- `Resize` en 255x255 pour réduire le nombre de paramètres,
- `Grayscale`,
- `ToTensor` pour transformer l'image en tenseur.

N'hésitez pas à chercher de l'aide dans la [documentation](https://pytorch.org/vision/0.9/transforms.html).

In [ ]:
transform = None

Utilisez la fonction `ImageFolder` en utilisant le path ou se trouve les données '/content/drive/MyDrive/chest_xray/train_small' et la fonction `transform`.

N'hésitez pas à regarder la [documentation](https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html).

In [ ]:
dataset_train = None

Vous n'allez pas charger directement toutes les images en mémoire pendant l'entraînement. Vous allez utilisez la fonction *flow_from_directory* pour renseigner l'endroit ou se trouve vos images. Les images seront lu batch par batch.

Utilisez :
- `dataset` ,
- un batch de 32 ,
- un shuffle à True.

N'hésitez pas à regarder la [documentation](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

In [ ]:
dataloader_train = None

Visualisation du générateur.

In [ ]:
for x, y in dataloader_train:
  print(x[0].shape)
  print(y[0])
  plt.imshow(x[0].permute(1, 2, 0))
  break

Faire de même pour le jeu de test et de validation.

In [ ]:
dataset_test = None

dataloader_test = None

# Initialisez votre modèle

Créez l'architecture suivante:
- Convolution avec 16 filtres de 3 de côté,
- Relu,
- Maxpooling avec 2 de côté et un stride de 2,
- Convolution avec 32 filtres de 3 de côté,
- Relu,
- Maxpooling avec 2 de côté et un stride de 2,
- Convolution avec 64 filtres de 3 de côté,
- Relu,
- Maxpooling avec 2 de côté et un stride de 2,
- Convolution avec 128 filtres de 3 de côté,
- Relu,
- Maxpooling avec 2 de côté et un stride de 2,
- Flatten,
- Linear avec 4608 données en entrée et 64 neurones,
- Relu,
- Linear avec 10 neurones,
- Softmax.

In [ ]:
model = None

Affiche l'architecture avec `print`

In [ ]:
None

Affiche le modèle en utilisant `summary`.

In [ ]:
None

# Les fonctions d'entraînement

Utilisez la fonction `save` pour sauver le modèle passé en paramètre au chemin indiqué en paramètre.

In [ ]:
def save_model(model, path):
    None

In [ ]:
def number_of_good_prediction(prediction:float, target:int):
  one_hot_prediction = np.argmax(prediction, axis=1)
  return np.sum(one_hot_prediction == target)

In [ ]:
def step(model:torch.nn.Sequential,
         opt:torch.optim,
         criterion:torch.nn.modules.loss,
         x_train:torch.Tensor,
         y_train:torch.Tensor,
         metric_function)->tuple:
  """
  Executes a single training step for a PyTorch model.
  This function performs a forward pass to compute the model's predictions, calculates
  the loss between predictions and actual target values, computes gradients for each
  model parameter, and updates the parameters using the optimizer.

  Args:
      model (torch.nn.Sequential): The PyTorch model to train.
      optimizer (torch.optim.Optimizer): Optimizer used to update the model's parameters.
      criterion (torch.nn.modules.loss._Loss): Loss function used to compute the error.
      x_train (torch.Tensor): Input training data (features).
      y_train (torch.Tensor): Ground truth labels or target values for the training data.
  Returns:
      tuple: The updated model and the computed loss for the current step.
  """

  # Réinitialisez les gradients d'optimizer à zéro avec la méthode 'zero_grad'
  opt.zero_grad()

  # Calculez les prédiction sur le jeu d'entraînement avec la méthode 'froward'
  prediction = model.forward(x_train)

  # Calculez l'erreur de prédiction avec 'criterion'
  loss = criterion(prediction, y_train)

  performance = metric_function(prediction.detach().numpy(), y_train.detach().numpy())

  # Calculez les gradients avec la méthode 'backward'
  loss.backward()

  # Mettre à jour les paramètres du modèle avec la méthode 'step'
  opt.step()

  return model, loss, performance

In [ ]:
def fit(model, optimizer, criterion, epoch, trainloader, testloader, metric_function):
    epoch = epoch
    history_train_loss = []
    history_test_loss = []
    history_train_metrics = []
    history_test_metrics = []

    reference_performance = 0

    for e in range(epoch) :

      train_loss_batch = 0
      test_loss_batch = 0
      train_metric_batch = 0
      test_metric_batch = 0

      for images, labels in trainloader:

        # mise à jour des poids avec la fonction 'step'
        model, train_loss, train_performance = step(model, optimizer, criterion, images, labels, metric_function)

        train_loss_batch += train_loss.detach().numpy()

        train_metric_batch += train_performance


      for images, labels in testloader:

        prediction = model.forward(images)

        test_loss = criterion(prediction, labels)

        test_metric_batch += metric_function(prediction.detach().numpy(), labels.detach().numpy())

        test_loss_batch += test_loss.detach().numpy()

      train_loss_batch /= len(trainloader.sampler)
      test_loss_batch /= len(testloader.sampler)

      train_metric_batch /= len(trainloader.sampler)
      test_metric_batch /= len(testloader.sampler)

      # Sauvegarde des coûts d'entraînement avec append
      history_train_loss = np.append(history_train_loss, train_loss_batch)
      history_test_loss = np.append(history_test_loss, test_loss_batch)

      # Sauvegarde des coûts d'entraînement avec append
      history_train_metrics = np.append(history_train_metrics, train_metric_batch)
      history_test_metrics = np.append(history_test_metrics, test_metric_batch)

      print('train_loss : '+str(np.squeeze(train_loss_batch))+ ' test_loss : '+str(np.squeeze(test_loss_batch)))
      print('train_metric : '+str(np.squeeze(train_metric_batch))+ ' test_metric : '+str(np.squeeze(test_metric_batch)))
      print('-------------------------------------------------------------------------------------------------')

      if test_metric_batch > reference_performance:
        reference_performance = test_metric_batch
        save_model(model, f'best_model_{test_metric_batch}.pth')

    return model, history_train_loss, history_test_loss, history_train_metrics, history_test_metrics


Initialisez `criterion` avec la fonction `NLLLoss`.

Initialisez `optimizer` avec `Adam` et un learning rate de 0.001.

In [ ]:
criterion = None
optimizer = None

# Entraînez votre modèle

In [ ]:
epoch = 10

Utilisez la fonction `fit` pour entraîner le modèle.

In [ ]:
model, history_train_loss, history_test_loss, history_train_metrics, history_test = None

# Visualisation des performances

In [ ]:
plt.plot(np.arange(epoch), history_train_loss, label='train loss')
plt.plot(np.arange(epoch), history_test_loss, label='test loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend(loc='upper left')
plt.legend()
plt.show()

In [ ]:
plt.plot(history_train_metrics, label='train accuracy')
plt.plot(history_test, label='test accuracy')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()